<a href="https://colab.research.google.com/github/sanu1230/Academic-Project/blob/master/Auto_ML_TPOT_Regression_Property_sales_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TPOT Regression - Property sales prediction

In [1]:
# Install tpot on the server
!pip install tpot

     |████████████████████████████████| 92kB 7.5MB/s 
     |████████████████████████████████| 163kB 12.4MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-cp36-none-any.whl size=11956 sha256=122b42382f5da8f2704ea03b10caa6a4aa9bee6b12a79f9ca4919a15000df669
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built stopit


## Import Packages

In [2]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# Import the tpot regressor
from tpot import TPOTRegressor

In [15]:
#Download dataset
!wget -q https://www.dropbox.com/s/6tc7e6rc395c7jz/nyc-property-sales.zip

In [16]:

#Unzip the data
!unzip nyc-property-sales.zip > /dev/null; echo " done."

 done.


In [17]:
!ls

nyc-property-sales.zip	nyc-rolling-sales.csv  sample_data


In [19]:
import warnings
warnings.filterwarnings("ignore")
import datetime
import pandas as pd               
import numpy as np
import pickle
import time


## Download and preprocess the dataset

In [20]:
t_start = time.time()
df_prop = pd.read_csv('nyc-rolling-sales.csv')
t_end = time.time()
print('pd.read_csv(): {} s'.format(t_end-t_start)) # time [s]
df_prop.head()

pd.read_csv(): 0.2600677013397217 s


,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,,10009,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,,10009,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,,10009,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,,10009,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,,10009,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


In [21]:
df_prop.shape

(84548, 22)

In [22]:
# Sample n rows
n = 10000
df_prop = df_prop.sample(n, random_state=1)
df_prop.shape

(10000, 22)

In [23]:
#This column has no significance other than being an iterator
del df_prop['Unnamed: 0']
#This column has no significant value
del df_prop['EASE-MENT']


In [24]:
df_prop['SALE PRICE'] = df_prop['SALE PRICE'].replace(' -  ',np.nan)
df_prop.dropna(inplace=True)

In [25]:
df_prop['LAND SQUARE FEET'] = df_prop['LAND SQUARE FEET'].replace(' -  ',np.nan)
df_prop['GROSS SQUARE FEET'] = df_prop['GROSS SQUARE FEET'].replace(' -  ',np.nan)

In [26]:
## Define a function impute_median and fill land square feet and gross square feet with median values
def impute_median(series):
    return series.fillna(series.median())

df_prop['LAND SQUARE FEET'] = df_prop['LAND SQUARE FEET'].transform(impute_median)
df_prop['GROSS SQUARE FEET'] = df_prop['GROSS SQUARE FEET'].transform(impute_median)

In [27]:
#Convert few column datatypes into appropriate ones for conserving memory

df_prop['TAX CLASS AT TIME OF SALE'] = df_prop['TAX CLASS AT TIME OF SALE'].astype('category')
df_prop['TAX CLASS AT PRESENT'] = df_prop['TAX CLASS AT PRESENT'].astype('category')
df_prop['LAND SQUARE FEET'] = pd.to_numeric(df_prop['LAND SQUARE FEET'], errors='coerce')
df_prop['GROSS SQUARE FEET']= pd.to_numeric(df_prop['GROSS SQUARE FEET'], errors='coerce')
df_prop['SALE PRICE'] = pd.to_numeric(df_prop['SALE PRICE'], errors='coerce')
df_prop['BOROUGH'] = df_prop['BOROUGH'].astype('category')

In [28]:
# Let's remove sale price with a nonsensically small dollar amount: $0 most commonly. 
# Since these sales are actually transfers of deeds between parties: for example, parents transferring ownership to their home to a child after moving out for retirement.

df_prop = df_prop[df_prop['SALE PRICE']!=0]

In [29]:
#Let's also remove observations that have gross square feet less than 400 sq. ft
#Let's also remove observations that have gross square feet less than 400 sq. ft
#Let's also remove observations that have sale price than 1000 dollars

df_prop = df_prop[df_prop['GROSS SQUARE FEET']>400]
df_prop = df_prop[df_prop['LAND SQUARE FEET']>400]
df_prop = df_prop[df_prop['SALE PRICE']>1000]

In [30]:
q = df_prop["SALE PRICE"].quantile(0.99)
q

15750000.0

In [31]:
df_prop = df_prop[df_prop["SALE PRICE"] < q]
df_prop_lin = df_prop.copy()

In [32]:
# Convert sale date into time,month,year and day
import datetime
df_prop['SALE DATE'] = pd.to_datetime(df_prop['SALE DATE'])
df_prop['year'] = df_prop['SALE DATE'].dt.year
df_prop['month'] = df_prop['SALE DATE'].dt.month
df_prop['day'] = df_prop['SALE DATE'].dt.day
df_prop['time'] = df_prop['SALE DATE'].dt.hour
df_prop['week_day'] = df_prop['SALE DATE'].dt.day_name()

In [33]:
#Assign numbered bouroughs to bourough names
dic = {1: 'Manhattan', 2: 'Bronx', 3: 'Brooklyn', 4: 'Queens', 5:'Staten Island'}
df_prop["borough_name"] = df_prop["BOROUGH"].apply(lambda x: dic[x])

In [34]:
df_prop.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,year,month,day,time,week_day,borough_name
32184,3,CANARSIE,01 ONE FAMILY DWELLINGS,1,8066,18,A2,1249 EAST 87 STREET,,11236,1,0,1,2833.0,1400.0,1935,1,A2,315270,2016-11-17,2016,11,17,0,Thursday,Brooklyn
83718,5,WEST NEW BRIGHTON,01 ONE FAMILY DWELLINGS,1,223,4,A5,447 CLOVE ROAD,,10310,1,0,1,1456.0,1120.0,1899,1,A5,112500,2016-09-08,2016,9,8,0,Thursday,Staten Island
56085,4,FLUSHING-NORTH,01 ONE FAMILY DWELLINGS,1,4808,11,A1,30-15 148TH STREET,,11354,1,0,1,6789.0,2971.0,1930,1,A1,1285000,2016-10-12,2016,10,12,0,Wednesday,Queens
55547,4,FAR ROCKAWAY,02 TWO FAMILY DWELLINGS,1,15614,60,B3,12-12 PLAINVIEW AVENUE,,11691,2,0,2,1950.0,1488.0,1940,1,B3,385000,2016-12-12,2016,12,12,0,Monday,Queens
80343,5,NEW BRIGHTON-ST. GEORGE,31 COMMERCIAL VACANT LAND,4,13,68,V1,208 RICHMOND TERRACE,,10301,0,0,0,5756.0,1644.5,0,4,V1,1200000,2017-01-12,2017,1,12,0,Thursday,Staten Island


In [35]:
#Dropping few columns
del df_prop_lin['BUILDING CLASS AT PRESENT']
del df_prop_lin['BUILDING CLASS AT TIME OF SALE']
del df_prop_lin['NEIGHBORHOOD']
del df_prop_lin['ADDRESS']
del df_prop_lin['SALE DATE']
del df_prop_lin['APARTMENT NUMBER']
del df_prop_lin['RESIDENTIAL UNITS']
del df_prop_lin['COMMERCIAL UNITS']


In [36]:
#Select the variables to be one-hot encoded
one_hot_features = ['BOROUGH', 'BUILDING CLASS CATEGORY','TAX CLASS AT PRESENT','TAX CLASS AT TIME OF SALE']
# Convert categorical variables into dummy/indicator variables (i.e. one-hot encoding).
one_hot_encoded = pd.get_dummies(df_prop_lin[one_hot_features],drop_first=True)
one_hot_encoded.info(verbose=True, memory_usage=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5873 entries, 32184 to 64852
Data columns (total 51 columns):
 #   Column                                                               Non-Null Count  Dtype
---  ------                                                               --------------  -----
 0   BOROUGH_2                                                            5873 non-null   uint8
 1   BOROUGH_3                                                            5873 non-null   uint8
 2   BOROUGH_4                                                            5873 non-null   uint8
 3   BOROUGH_5                                                            5873 non-null   uint8
 4   BUILDING CLASS CATEGORY_02 TWO FAMILY DWELLINGS                      5873 non-null   uint8
 5   BUILDING CLASS CATEGORY_03 THREE FAMILY DWELLINGS                    5873 non-null   uint8
 6   BUILDING CLASS CATEGORY_04 TAX CLASS 1 CONDOS                        5873 non-null   uint8
 7   BUILDING CLASS CATE

In [37]:
# Replacing categorical columns with dummies
fdf = df_prop_lin.drop(one_hot_features,axis=1)
fdf = pd.concat([fdf, one_hot_encoded] ,axis=1)


In [38]:
fdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5873 entries, 32184 to 64852
Data columns (total 59 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   BLOCK                                                                5873 non-null   int64  
 1   LOT                                                                  5873 non-null   int64  
 2   ZIP CODE                                                             5873 non-null   int64  
 3   TOTAL UNITS                                                          5873 non-null   int64  
 4   LAND SQUARE FEET                                                     5873 non-null   float64
 5   GROSS SQUARE FEET                                                    5873 non-null   float64
 6   YEAR BUILT                                                           5873 non-null   int64  
 7   S

### Standardization

In [39]:
#Standardize rows into uniform scale

X = fdf.drop(['SALE PRICE'],axis=1)
y = fdf['SALE PRICE']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)

# Scale and center the data
fdf_normalized = scaler.transform(X)

# Create a pandas DataFrame
X = pd.DataFrame(data=fdf_normalized, index=X.index, columns=X.columns)





### Split the data into train and test

In [40]:
from sklearn.model_selection import train_test_split   #splitting data
from pylab import rcParams
from sklearn.linear_model import LinearRegression         #linear regression
from sklearn.metrics.regression import mean_squared_error #error metrics
from sklearn.metrics import mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(fdf_normalized,y)

In [41]:
#@title Save 
df_prop1 = df_prop.copy()

In [42]:
#@title Restore
df_prop = df_prop1.copy()

In [43]:
# Convert to numpy arrays
training_features = np.array(X_train)


# Sklearn wants the labels as one-dimensional vectors
training_targets = np.array(y_train).reshape((-1,))


## Model Building

In [44]:
#100 pipelines
#5 generations
# Create a tpot object with a few parameters
tpot = TPOTRegressor(generations=5, population_size= 100, 
                    max_time_mins = 10,
                    n_jobs = -1, verbosity = 2, cv = 5)

In [45]:
# Fit the tpot model on the training data
tpot.fit(training_features, training_targets)


10.38 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: ExtraTreesRegressor(input_matrix, bootstrap=False, max_features=0.8, min_samples_leaf=3, min_samples_split=12, n_estimators=100)


TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=5,
              disable_update_check=False, early_stop=None, generations=5,
              log_file=<ipykernel.iostream.OutStream object at 0x7f6b26597978>,
              max_eval_time_mins=5, max_time_mins=10, memory=None,
              mutation_rate=0.9, n_jobs=-1, offspring_size=None,
              periodic_checkpoint_folder=None, population_size=100,
              random_state=None, scoring=None, subsample=1.0, template=None,
              use_dask=False, verbosity=2, warm_start=False)

In [46]:
# Show the final model
print(tpot.fitted_pipeline_)

Pipeline(memory=None,
         steps=[('extratreesregressor',
                 ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0,
                                     criterion='mse', max_depth=None,
                                     max_features=0.8, max_leaf_nodes=None,
                                     max_samples=None,
                                     min_impurity_decrease=0.0,
                                     min_impurity_split=None,
                                     min_samples_leaf=3, min_samples_split=12,
                                     min_weight_fraction_leaf=0.0,
                                     n_estimators=100, n_jobs=None,
                                     oob_score=False, random_state=None,
                                     verbose=0, warm_start=False))],
         verbose=False)


In [47]:
# Export the pipeline as a python script file
tpot.export('tpot_exported_pipeline.py')

## Evaluating final model

In [48]:
 # Evaluate the final model
print(tpot.score(X_test, y_test))

-1171518603690.3508
